In [1]:
pip install transformers torch sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import datasets
import csv
import os
import pandas as pd
from datasets import concatenate_datasets,Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,MBartForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer,TrainingArguments, Trainer

In [4]:
dataset_file = '/kaggle/input/clear-data/overall_80K_corrected.csv'
dataset = pd.read_csv(dataset_file).iloc[:, 2:]


In [5]:
dataset = dataset.sample(frac=0.125)
dataset

,target,source
35872,А̄ги олн хӯп о̄ньщыс.,Девушка имела серебряную ложку.
44300,"А̄щēн ла̄ваве: ""Наӈ турманланэ̄й во̄ртӣул ёмен.""","Отец говоритӣему однажды: ""Тыв лесу не ходи до..."
34325,"Ты сака ка̄ста̄л рӯпата, ос тав щар та̄рвитыӈ ...","Это очень скучная работа, но совсем не трудная."
65585,Хус арыгтем та̄рвит ва̄рмаль осн-паттувес.,Двадцать с лишним проблемных дел выявили.
30799,Ам то̄ванакт морых атыглэ̄гум.,Я иногда морошку собираю.
...,...,...
55893,Та̄н ань олныл щар ат мивет.,Им сейчас совсем не выплачивают деньги.
35044,Ӯлмыл а̄гикве яныгмēгын.,Во сне ты доченька растёшь.
29729,Тувыл сāт ōсься колас пумтар сāт ёмас колас пу...,Также семь тощих колосьев поглотили семь хорош...
38386,Щēрки сасгум э̄кватэ На̄таля о̄лыс.,Сергей был женат на Наталье.


In [6]:
train_index = int(len(dataset)*0.9)
train_dataset=dataset[:train_index]
val_dataset=dataset[train_index:]

In [7]:
val_dataset.shape

(7996, 2)

In [7]:
model_ckpt = "/kaggle/input/model-mbart"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = MBartForConditionalGeneration.from_pretrained(model_ckpt)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [8]:
for name, param in model.named_parameters():
    if 'encoder' in name or ('decoder.layer' in name and 'decoder.layer.11' not in name):  # Пример для MBart с 12 слоями
        param.requires_grad = False

In [9]:
train_data = Dataset.from_pandas(train_dataset)
val_data = Dataset.from_pandas(val_dataset)

In [10]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["source"], max_length=1024, padding="max_length",truncation=True)

    #with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch["target"], max_length=1024, padding="max_length", truncation=True)

    return {"input_ids": input_encodings["input_ids"],
           "attention_mask": input_encodings["attention_mask"],
           "labels": target_encodings["input_ids"]}

train_dataset_tf = train_data.map(convert_examples_to_features, batched=True, remove_columns=["source","target"])
val_dataset_tf = val_data.map(convert_examples_to_features, batched=True, remove_columns=["source","target"])

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
train_dataset_tf

In [11]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
training_args = Seq2SeqTrainingArguments(output_dir='mbartTrans', num_train_epochs=2,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                evaluation_strategy='steps',save_strategy='no',eval_steps=1000,logging_steps=1000,
                                  weight_decay=0.01, push_to_hub=False,
                           fp16=True,learning_rate=2e-4,optim="adafactor")

In [13]:
import gc
gc.collect()

85

In [14]:
trainer = Seq2SeqTrainer(model=model, args=training_args, tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=train_dataset_tf,
                        eval_dataset=val_dataset_tf)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1000,0.063300,0.063310



KeyboardInterrupt



In [ ]:
trainer.save_model("mbartfortranslation5000") 

In [ ]:
import shutil
shutil.make_archive('mbartfortranslation', 'zip', '/kaggle/working/mbartfortranslation5000')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sacrebleu

# Загрузка токенайзера и модели
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/model-mbart")  # путь к вашей модели
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/model-mbart")





In [7]:
import sacrebleu
from tqdm import tqdm

# Загрузка тестовых данных
df = pd.read_csv('/kaggle/input/clear-data/overall_80K_corrected.csv')
references = df.iloc[10000:10100]['target'].tolist()  # Преобразуем в список строк
inputs = df.iloc[10000:10100]['source'].tolist()

# Генерация предсказаний модели с tqdm для отображения прогресса
hypotheses = []
for input_sentence in tqdm(inputs, desc="Генерация гипотез"):
    inputs_tokenized = tokenizer(input_sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs_tokenized, max_length=128)  # max_length можно настроить
    generated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    hypotheses.append(generated_sentence)

# Расчет BLEU с помощью sacrebleu
bleu = sacrebleu.corpus_bleu(hypotheses, [references])  # Обернули references в список
print(f"BLEU score: {bleu.score}")


Генерация гипотез: 100%|██████████| 100/100 [05:46<00:00,  3.46s/it]

BLEU score: 7.088144222465497
